# Extract 2D obstacles
Generate polygons for all static obstacles, and merge these with the sidewalk polygons.

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pandas as pd
import geopandas as gpd
import ast
import pathlib
import shapely.geometry as sg
import shapely.ops as so
from tqdm.notebook import tqdm
tqdm.pandas()

from upcp.utils import bgt_utils
from upcp.utils import csv_utils
from upcp.utils import las_utils

from upc_sw.cluster2polygon import Cluster2Polygon
from upc_sw import sw_utils
from upc_sw import poly_utils

## Create polygons of static obstacles
In the previous notebook, we performed a change detection algorithm that calculated M3C2 distance for each point in the point cloud. Based on negative and positive threshold values we can filter for the static points in the point cloud. We then cluster these into individual obstacles and create bounding polygons for each.

In [ ]:
### SETTINGS ###

CRS = 'epsg:28992'

pc_data_folder = '../datasets/pointclouds/'
out_folder = '../datasets/output/'

# Scraped sidewalk and terras data for the area (see Notebook 1c)
sidewalk_data = '../datasets/bgt/bgt_voetpad.gpkg'
terras_data = '../datasets/bgt/terras_data.gpkg'

# Output file
obstacle_data = f'{out_folder}obstacles.gpkg'

# Distance threshold for static obstacles.
m3c2_threshold = 0.2

# Convert 3D Obstacle blobs to 2D polygons using a clustering algorithm.
# Set use_concave=False to use the faster convex hull.
# Change alpha to determine the 'concaveness' of the concave hull, with 0 being convex.
c2p = Cluster2Polygon(min_component_size=100, grid_size=0.2, use_concave=True, concave_min_area=2.5, alpha=0.5)

# Create output folder if it doesn't exist
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}m3c2/')

In [ ]:
tile_tqdm = tqdm(all_tiles, unit='tile', smoothing=0)

obstacle_gdf = gpd.GeoDataFrame(columns=['tilecode', 'type', 'geometry'], geometry='geometry', crs=CRS)

for tilecode in tile_tqdm:
    tile_tqdm.set_postfix_str(tilecode)
    
    # Read point cloud with M3C2 distances
    in_file = f'{pc_data_folder}m3c2/m3c2_{tilecode}.laz'
    points, m3c2_distance = sw_utils.read_las(in_file, extra_val='M3C2_distance', extra_val_dtype='float32')

    # Filter for static points
    mask = np.abs(m3c2_distance) < m3c2_threshold
    
    if np.count_nonzero(mask) > 0:
        # Get the polygons
        polygons, types = c2p.get_obstacle_polygons(points[mask])
        data = {'tilecode': [tilecode]*len(polygons),
                'type': types,
                'geometry': polygons}
        obstacle_gdf = obstacle_gdf.append(gpd.GeoDataFrame(data, geometry='geometry', crs=CRS))

In [ ]:
# Fix invalid polygons
obstacle_gdf['geometry'] = obstacle_gdf['geometry'].progress_apply(poly_utils.fix_invalid)

In [ ]:
# Save the obstacle GeoDataFrame.
if len(obstacle_gdf) > 0:
    obstacle_gdf.to_file(obstacle_data, driver='GPKG')
else:
    print('No obstacle data to write.')

## Merge sidewalk polygons with obstacles
The found obstacles (polygons) in the previous step are merged with the sidewalk polygons as interiors.

In [ ]:
### SETTINGS ###

merged_output_file = f'{out_folder}sidewalks_with_obstacles.gpkg'

# Buffer width around the sidewalk to preserve its shape. Set to '0' to ignore this.
sw_buffer = 0.01

# Add padding around obstacles.
obstacle_padding = 0.05

In [ ]:
# Load the sidewalk data
sidewalk_gdf = gpd.read_file(sidewalk_data, crs=CRS).set_index('ogc_fid')

# Load the "terras" data
terras_gdf = gpd.read_file(terras_data, crs=CRS).set_index('id')

# Load the obstacle data
obstacle_gdf = gpd.read_file(obstacle_data, crs=CRS)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1)

sidewalk_gdf.plot(ax=ax, edgecolor='black')
terras_gdf.plot(ax=ax, color='orange')
obstacle_gdf.plot(ax=ax, color='grey')

In [ ]:
def merge_obstacles(row):
    # Subtract all obstacles that intersect the sidewalk polygon.
    sw_poly = row.geometry
    obst_poly = (obstacle_gdf[obstacle_gdf.intersects(sw_poly)]
                 .buffer(obstacle_padding)
                 .unary_union)
    # TODO buffer also for terrace shapes?
    terras_poly = terras_gdf[terras_gdf.intersects(sw_poly)].unary_union
    merged_poly = so.unary_union([poly for poly in [obst_poly, terras_poly] if poly is not None])
    # TODO: do something with obstacle type?
    if merged_poly is not None:
        sw_poly = sw_poly.buffer(sw_buffer) - sw_poly.intersection(merged_poly)
    # if type(sw_poly) == sg.MultiPolygon:
    #     # TODO: this shouldn't happen anymore, delete?
    #     parts = [p for p in sw_poly.geoms if p.area > 1] # TODO: magic number
    #     if len(parts) > 1:
    #         print('Warning, sidewalk broken into parts.')
    #         sw_poly = sg.MultiPolygon(parts)
    #     else:
    #         sw_poly = parts[0]
    return sw_poly

sw_merged_gdf = sidewalk_gdf.copy()
sw_merged_gdf['geometry'] = sidewalk_gdf.progress_apply(merge_obstacles, axis=1)

In [ ]:
# Save the merged sidewalk data.
sw_merged_gdf.to_file(merged_output_file, driver='GPKG')